# Some analysis of the Swedish archive of conscripts

In [106]:
#ESIM	STRING	Ej simkunnig
#LASV	STRING	Läs-skrivsvårighet. 1=läs och skrivsvårigheter. F r o m 1984.
#PENR	STRING	Personnummer
#INST	STRING	Inställelsedatum
#PPRF_BEFL	STRING	Befälslämplighet
#LNGD	STRING	Längd i cm
#MASS	STRING	Vikt i kg
codes_to_read = ['ESIM', 'LASV', 'PENR', 'INST', 'PPRF_BEFL', 'LNGD', 'MASS']
import os.path
import numpy as np
import time

INSARK_DB = "insark_db.npz"
if os.path.exists(INSARK_DB):
    data = np.load(INSARK_DB)
else:
    print("Loading data...", end="")
    t = time.time()
    data = {}
    for code in codes_to_read:
        data[code] = list()
    def to_int(d):
        try:
            if len(d)>0:
                return int(d)
        except:
            pass
        return None
    with open("INSARK.csv") as datafile:
        codes = datafile.readline().split('\t')
        codes[-1] = codes[-1].strip('\n')
        for linenumber, textline in enumerate(datafile):
            #print(textline)
            split_line = textline.split('\t')
            for code in codes_to_read:
                d = split_line[codes.index(code)].strip()
                if code in ['LNGD', 'MASS', 'ESIM', 'LASV', 'PPRF_BEFL', 'INST']:
                    d = to_int(d)
                data[code].append(d)
            #if linenumber > 100000:
            #    break
    # Clean up the data
    data['PENR'] = [to_int(e[:4]) for e in data['PENR']]
    mask = [data['LNGD'][i] is not None and data['MASS'][i] is not None for i in range(len(data['LNGD']))]
    mask = [mask[i] and data['LNGD'][i]>0 and data['MASS'][i]>0 for i in range(len(mask))]
    mask = [mask[i] and data['PENR'][i] is not None for i in range(len(mask))]
    for code in codes_to_read:
        data[code] = [e for i, e in enumerate(data[code]) if mask[i]]   
    data['LNGD'] = np.asarray(data['LNGD'], dtype=np.int)
    data['MASS'] = np.asarray(data['MASS'], dtype=np.int)
    data['PENR'] = np.asarray(data['PENR'], dtype=np.int)
    data['INST'] = np.asarray([0 if e is None else e for e in data['INST']], dtype=np.int)
    data['ESIM'] = np.asarray([0 if e is None else e for e in data['ESIM']], dtype=np.bool)
    data['LASV'] = np.asarray([0 if e is None else e for e in data['LASV']], dtype=np.bool)
    data['PPRF_BEFL'] = np.asarray([-1 if e is None else e for e in data['PPRF_BEFL']], dtype=np.int)

    np.savez_compressed(INSARK_DB, **data)
    print("done (%.1fs)" % (time.time()-t))

Loading data...done (34.1s)


In [110]:
length = data['LNGD']
weight = data['MASS']
born = data['PENR']//100

array([13, 13, 13, ..., 79, 79, 79])